# Estimating properties of LFP-Li batteries

## 1. Challenge laid down

Dennis Kopljar suggested, in a tweet below, a nice exercise with this tool in which we look at what energy densities might be realistic for hypothetical battery cells with an LFP cathode and a Li metal anode. This follows recent articles by the journalist Steve Levine discussing the growing interest in this possibility, and the solid-state company QuantumScape also announcing they were looking into this.

<blockquote class="twitter-tweet"><p lang="en" dir="ltr">First community exercise 😬<br>What energy densities can be achieved / are realistic with this approach? <a href="https://t.co/U08s5kPA3G">https://t.co/U08s5kPA3G</a></p>&mdash; Dennis Kopljar 🔋⚡ (@DennisKopljar) <a href="https://twitter.com/DennisKopljar/status/1427952947384700932?ref_src=twsrc%5Etfw">August 18, 2021</a></blockquote> <script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>

As it has turned out, this notebook covers a lot of possibilities/hacks with this code, from Li metal anodes, to solid state electrolytes, to anode-free, but also batch calculations with `for` loops. Let's get stuck in.

## 2. Get started

I recommend reading the quickstart notebook if you haven't already.

In [1]:
# Uncomment this line if the cellmodel.jl file is not already in the working folder.
# download("https://raw.githubusercontent.com/mjlacey/cellmodels/master/cellmodel.jl", "cellmodel.jl")

# Include the cellmodel.jl file it to load the required packages and functions.
include("cellmodel.jl")

volumetric_energy (generic function with 3 methods)

## 3. Model building

## 3.1 Material definitions

Let's define the materials we want to use. Most of these are defined already in the Quickstart notebook, but the new one we will introduce is LFP. For the purposes of this exercise, I am estimating an LFP material with a specific capacity of 165 ± 5 mAh/g and an average discharge potential of 3.375 ± 0.025 V. For this exercise we will compare it with the state-of-the-art layered oxide cathode, NMC811, where I have estimated 195 ± 5 mAh/g and an average potential of 3.86 V under practical conditions, from the literature.

In [2]:
# Positive electrodes
NMC811 = ActiveMaterial("NMC811", 195 ± 5, 3.86)    # NMC811 determined from literature
LFP = ActiveMaterial("LFP", 165 ± 5, 3.375 ± 0.025) # LFP capacities and potential estimated

# Negative electrodes
Gr = ActiveMaterial("Graphite", 344 ± 8.6, 0.17)    # Gr determined from literature

# Current collectors
Al14 = CurrentCollector("Al", 14E-4) # 14 µm Al
Cu8 = CurrentCollector("Cu", 8E-4)   # 8 µm Al

# Electrolytes
LP40 = Electrolyte(salt = "LiPF6", solvent = "EC:DEC", concentration = 1.1 ± 0.1)

# Separator
PE_Al12 = Separator(name = "porous PE + alumina", thickness = 12E-4, porosity = 0.44, density = 1.18)

Separator
  name: String "porous PE + alumina"
  thickness: Float64 0.0012
  porosity: Float64 0.44
  density: Float64 1.18


## 3.2 Cell model

Here we will use the `PouchCell` model. This is of course defined a bit differently from the cylindrical cell. It assumes a rectangular, stacked cell, with 0.5 mm thick tabs of Al and Ni to contact the electrodes. At the moment, the `PouchCell` can only be defined with a set number of layers, rather than a maximum thickness, unlike the other models which all fit the electrodes into a constrained space. I will likely update this later, but for the moment this results in cells which can vary in thickness depending on the materials used.

The model is constructed with electrodes similar to those used in the Quickstart guide. We are assuming a typical automotive-size format, 30 x 10 cm, and 35 layers, which should give us a thickness around 1 cm. The pouch is assumed to be 113 µm thick Al/polymer laminate with an average density of 1.8 g/cm, and the pouch extending 1 cm from the jelly roll on the long and wide sides. The terminals are 2 x 5 cm each, and we are assuming 10 ± 10 g of unaccounted-for mass.

In [3]:
cell = PouchCell(
    name = "A benchmark NMC811 automotive pouch cell",
    positive = Electrode(
        ElectrodeComposite(
            active_material = NMC811, 
            areal_cap = 3.3 ± 0.1, 
            active_frac = 0.95 ± 0.02, 
            density = 3.5 ± 0.1),
        Al14
    ),
    negative = Electrode(
        ElectrodeComposite(
            active_material = Gr, 
            areal_cap = (3.3 ± 0.1) * (1.1 ± 0.03), 
            active_frac = 0.965 ± 0.01, 
            density = 1.6 ± 0.1),
        Cu8
    ),
    separator = PE_Al12,
    electrolyte = LP40,
    ecap_ratio = 1.7 ± 0.1,
    
    height = 30.0, width = 10.0, nlayers = 35,
    pouchthickness = 113E-4,
    pouchdensity = 1.8,
    pouchclearance = 1.0,
    termh = 2.0, termw = 5.0,
    extramass = 10 ± 10,
    llifactor = 0.93 ± 0.02
)

A benchmark NMC811 automotive pouch cell
NMC811 cathode @ 3.3 ± 0.1 mAh/cm2
64.4 ± 2.4 Ah, 237.8 ± 8.8 Wh
925.0 ± 26.0 g
257.2 ± 8.3 Wh/kg
634.0 ± 32.0 Wh/L


This gives us a 64.4 Ah cell (and with this definition of cell, every subsequent cell here will also be 64.4 Ah), with an energy density of 257 Wh/kg and 634 Wh/L. This sounds pretty sensible, so we can continue.

## 3.3 Cell model function

From here on we want to compare LFP with NMC811, with some different tweaks to the cell model, so we will want to make several different cell models. To make this easier, we can make a simple function, let's call it `automotive_pouch()`, based on the previous code cell where we can change individual parameters at a time.

In [4]:
function automotive_pouch(;
        name = "An example automotive pouch cell", 
        positive = Electrode(
            ElectrodeComposite(
                active_material = NMC811, 
                areal_cap = 3.3 ± 0.1, 
                active_frac = 0.95 ± 0.02, 
                density = 3.5 ± 0.1),
            Al14
        ), 
        negative = Electrode(
            ElectrodeComposite(
                active_material = Gr, 
                areal_cap = (3.3 ± 0.1) * (1.1 ± 0.03), 
                active_frac = 0.965 ± 0.01, 
                density = 1.6 ± 0.1),
            Cu8
        ), 
        separator = PE_Al12,
        electrolyte = LP40, 
        ecap_ratio = 1.7 ± 0.1,
        height = 30.0, width = 10.0, nlayers = 35, 
        pouchthickness = 113E-4, 
        pouchdensity = 1.8, 
        pouchclearance = 1.0, 
        termh = 2.0, termw = 5.0, 
        extramass = 10 ± 10, llifactor = 0.93 ± 0.02)
        
    cell = PouchCell(
        name = name,
        positive = positive,
        negative = negative,
        separator = separator,
        electrolyte = electrolyte,
        ecap_ratio = ecap_ratio,
    
        height = height, width = width, nlayers = nlayers,
        pouchthickness = pouchthickness,
        pouchdensity = pouchdensity,
        pouchclearance = pouchclearance,
        termh = termh, termw = termw,
        extramass = extramass,
        llifactor = llifactor
    )
    
    return cell
end

# Now let's check it works
automotive_pouch()

An example automotive pouch cell
NMC811 cathode @ 3.3 ± 0.1 mAh/cm2
64.4 ± 2.4 Ah, 237.8 ± 8.8 Wh
925.0 ± 26.0 g
257.2 ± 8.3 Wh/kg
634.0 ± 32.0 Wh/L


## 3.4 Batch processing

To compare LFP with NMC811 in these different situations I will use the previous function in a `for` loop with some of the properties I want to change in some sort of array I can refer to. The main ones I want to change for now are the materials themselves, and the electrode density, to reflect the different densities of the materials. For this, I'm going to assume NMC811 has a bulk density of 5 g/cc, and for LFP 3.6 g/cc, and that both can be calendered to a porosity of 27-33% compared to the theoretical bulk density. I can put this information in a `DataFrame`, like so:

In [5]:
positives = DataFrame(
    material = [NMC811, LFP],
    density = [5 * (0.7 ± 0.03), 3.6 * (0.7 ± 0.03)]
)

,material,density
,ActiveM…,Measure…
1,"ActiveMaterial(""NMC811"", 195.0±5.0, 3.86)",3.5±0.15
2,"ActiveMaterial(""LFP"", 165.0±5.0, 3.375±0.025)",2.52±0.11


In a `for` loop I can now make LFP and NMC811 cell models and pick certain properties of interest to output in a table:

In [6]:
result = DataFrame(material = String[], cell_energy = Number[], cell_mass = Number[], 
    grav_energy = Number[], vol_energy = Number[], electrode_thickness = Number[], cell_thickness = Number[])

for i in 1:nrow(positives)
    
    cell = automotive_pouch(
        positive = Electrode(
            ElectrodeComposite(
                active_material = positives[i,1], 
                areal_cap = 3.3 ± 0.1, 
                active_frac = 0.95 ± 0.02, 
                density = positives[i,2]),
            Al14
        )
    )
    
    push!(result, [positives[i,1].name, cell.energy, mass(cell), 
            gravimetric_energy(cell), volumetric_energy(cell), 
            thickness(cell.positive.composite) * 1E4, thickness(cell)])
    
end

println(result)

2×7 DataFrame
 Row │ material  cell_energy  cell_mass   grav_energy  vol_energy  electrode_thickness  cell_thickness 
     │ String    Number       Number      Number       Number      Number               Number         
─────┼─────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ NMC811      237.8±8.8  925.0±26.0    257.2±8.3  634.0±33.0             50.9±3.2     1.033±0.045
   2 │ LFP         206.6±7.8  993.0±30.0    208.1±7.0  451.0±22.0             83.5±5.4     1.261±0.054


So, the results of comparing LFP with NMC811, all other parameters being the same, gives the LFP cell with an energy density of 208 Wh/kg, 451 Wh/L. These numbers are fairly similar to some of the state of the art LFP cells coming out now, so I think we can treat these as reasonable as well. The LFP cell has a bit lower energy as a result of the lower average voltage, and the lower density and capacity of LFP means that to reach 3.3 mAh/cm2, the electrode needs to be 84 µm thick compared to 51 µm for the NMC811. This makes the cell thicker, so the ratio of the volumetric energy to gravimetric is lower for LFP.

# 4. Li metal

Let's try and calculate some hypothetical Li metal cells. We can perhaps start by defining Li metal as an `ActiveMaterial` with a theoretical capacity of 3862 mAh/g, and a potential of 0.00 V (if we reference everything on the Li scale).

In [7]:
# Li metal 
Li_metal = ActiveMaterial("Li metal", 3862, 0.00)

ActiveMaterial
  name: String "Li metal"
  spec_cap: Int64 3862
  avg_E: Float64 0.0


We can make another basic cell, where we assume the `ElectrodeComposite` is 100% Li metal with a density of 0.534 g/cc (the density of Li metal). At the moment, it's not possible to define the electrode by thickness alone, so here I have given the areal capacity as 10.32 mAh/cm2, which I know corresponds to about 50 µm thick Li foil, in theory. We're going to assume we're still using the same 8 µm Cu foil current collector.

In [8]:
cell = PouchCell(
    name = "A benchmark NMC811/Li metal cell",
    positive = Electrode(
        ElectrodeComposite(
            active_material = NMC811, 
            areal_cap = 3.3 ± 0.1, 
            active_frac = 0.95 ± 0.02, 
            density = 3.5 ± 0.1),
        Al14
    ),
    negative = Electrode(
        ElectrodeComposite(
            active_material = Li_metal,          # <----
            areal_cap = 10.32,                   # <---- I know from calculations, 50 µm Li ~= 10.3 mAh/cm2
            active_frac = 1,                     # <---- 100% Li metal
            density = 0.534),                    # <---- Density of Li metal
        Cu8
    ),
    separator = PE_Al12,
    electrolyte = LP40,
    ecap_ratio = 1.7 ± 0.1,
    
    height = 30.0, width = 10.0, nlayers = 35,
    pouchthickness = 113E-4,
    pouchdensity = 1.8,
    pouchclearance = 1.0,
    termh = 2.0, termw = 5.0,
    extramass = 10 ± 10,
    llifactor = 0.93 ± 0.02
)

A benchmark NMC811/Li metal cell
NMC811 cathode @ 3.3 ± 0.1 mAh/cm2
64.4 ± 2.4 Ah, 248.8 ± 9.2 Wh
746.0 ± 24.0 g
333.0 ± 10.0 Wh/kg
761.0 ± 25.0 Wh/L


This gives us an NMC811 cell with an energy density of 333 Wh/kg (30% more than graphite), 761 Wh/L (20% more than graphite). We can also check at this point that the thickness of the Li metal is what I think it is:

In [9]:
thickness(cell.negative.composite) * 1E4

50.04102200059351

Now we can once again make a `for` loop to compare LFP and NMC811, and pick out some interesting cell properties:

In [10]:
result = DataFrame(material = String[], grav_energy = Number[], vol_energy = Number[], 
    cell_thickness = Number[])

for i in 1:nrow(positives)
    
    cell = automotive_pouch(
        positive = Electrode(
            ElectrodeComposite(
                active_material = positives[i,1], 
                areal_cap = 3.3 ± 0.1, 
                active_frac = 0.95 ± 0.02, 
                density = positives[i,2]),
            Al14
        ),
        
        negative = Electrode(
            ElectrodeComposite(
                active_material = Li_metal,          # <----
                areal_cap = 10.32,                   # <---- I know from calculations, 50 µm Li ~= 10.3 mAh/cm2
                active_frac = 1,                     # <---- 100% Li metal
                density = 0.534),                    # <---- Density of Li metal
            Cu8
        )
    )
    
    push!(result, [positives[i,1].name, gravimetric_energy(cell), volumetric_energy(cell),
            thickness(cell)])
    
end

println(result)

2×4 DataFrame
 Row │ material  grav_energy  vol_energy  cell_thickness 
     │ String    Number       Number      Number         
─────┼───────────────────────────────────────────────────
   1 │ NMC811     333.0±10.0  761.0±27.0     0.901±0.022
   2 │ LFP         267.1±8.9  531.0±21.0     1.129±0.038


This gives us LFP cell with an energy density of 267 Wh/kg (+4% vs NMC811/Gr), 531 Wh/L (-16% vs NMC811/Gr) - so a like-for-like LFP/50 µm Li cell has a similar energy density to NMC811/Gr, but a lower volumetric energy density.

# 5. Li metal-solid state

We are also interested in solid state, since as was mentioned companies such as QuantumScape have indicated they are looking into it. We can consider two concepts which are perhaps of highest interest right now - an oxide based separator, such as LLZO, with a liquid catholyte, or a completely all-solid cell with a sulfide electrolyte.

We'll start with the LLZO. Let's assume that the separator has to be at least 20 µm thick, and LLZO has a density of 5.1 g/cc:

In [11]:
LLZO = Separator(name = "LLZO", thickness = 20E-4, porosity = 0, density = 5.1)

Separator
  name: String "LLZO"
  thickness: Float64 0.002
  porosity: Int64 0
  density: Float64 5.1


For now we'll keep it simple and assume we're still using the same 50 µm Li metal foil.

The complicated thing here is what to do with the electrolyte, or catholyte, in this case. We are assuming that there is no liquid in the separator, or on the anode side, but there is some in the cathode. We can do some more complicated calculations to work out the pore volume of the electrode, but for now let's just assume that we can take the electrolyte volume from the previous cells and divide it by 3.

(Note: I have previously worked out the pore volume for similar electrodes, and it comes out fairly close to this approximation, so we'll roll with it.)

In [12]:
result = DataFrame(material = String[], grav_energy = Number[], vol_energy = Number[], 
    cell_thickness = Number[])

for i in 1:nrow(positives)
    
    cell = automotive_pouch(
        positive = Electrode(
            ElectrodeComposite(
                active_material = positives[i,1], 
                areal_cap = 3.3 ± 0.1, 
                active_frac = 0.95 ± 0.02, 
                density = positives[i,2]),
            Al14
        ),
        
        negative = Electrode(
            ElectrodeComposite(
                active_material = Li_metal,         
                areal_cap = 10.32,                  
                active_frac = 1,                    
                density = 0.534),                   
            Cu8
        ),
        
        separator = LLZO,                 # <-----
        ecap_ratio = (1.7 ± 0.1) * 0.33   # <-----
    )
    
    push!(result, [positives[i,1].name, gravimetric_energy(cell), volumetric_energy(cell),
            thickness(cell)])
    
end

println(result)

2×4 DataFrame
 Row │ material  grav_energy  vol_energy  cell_thickness 
     │ String    Number       Number      Number         
─────┼───────────────────────────────────────────────────
   1 │ NMC811      296.1±9.4  716.0±25.0     0.957±0.022
   2 │ LFP         239.5±8.0  505.0±20.0     1.185±0.038


This now gives us an NMC811 cell with 296 Wh/kg and 716 Wh/L, vs 240 Wh/kg and 505 Wh/L for LFP. Lower than the liquid electrolyte, especially in gravimetric thanks to the much heavier separator, but a little lower in volumetric since the separator is a bit thicker.

# 6. Anode free

Now, we also want to consider the 'anode free' case, in which the only Li metal used in the cell is that which can be de-intercalated out of the cathode and plated onto the current collector. The model is not really set up for this, but I hack it in the following way.

First - I make a Li anode with an n/p ratio of 1 (i.e., the same capacity of anode as cathode). This will also give me the theoretical thickness of the cell when it is fully charged (i.e., all the Li is plated on the anode as the metal).

However - I don't want that Li metal included in my gravimetric energy calculation, because it 'belongs' to the cathode. So I can just make a new function which re-calculates the energy density, using the mass of the cell minus the mass of all the Li metal:

In [13]:
function gravimetric_energy_anodefree(cell::PouchCell)
    return 1000 * cell.energy / (mass(cell) - (2 * mass(cell.negative.composite) * cell.jr_area))
end

gravimetric_energy_anodefree (generic function with 1 method)

Now, if I include these changes in the model, and that new function, and run it:

In [14]:
result = DataFrame(material = String[], grav_energy = Number[], vol_energy = Number[], 
    cell_thickness = Number[])

for i in 1:nrow(positives)
    
    cell = automotive_pouch(
        positive = Electrode(
            ElectrodeComposite(
                active_material = positives[i,1], 
                areal_cap = 3.3 ± 0.1, 
                active_frac = 0.95 ± 0.02, 
                density = positives[i,2]),
            Al14
        ),
        
        negative = Electrode(
            ElectrodeComposite(
                active_material = Li_metal,          
                areal_cap = 3.3 ± 0.1,          # <-------            
                active_frac = 1,                    
                density = 0.534),                
            Cu8
        ),
        
        separator = LLZO,
        ecap_ratio = (1.7 ± 0.1) * 0.33
    )
    
    push!(result, [positives[i,1].name, gravimetric_energy_anodefree(cell), # <---- 
            volumetric_energy(cell), thickness(cell)])
    
end

println(result)

2×4 DataFrame
 Row │ material  grav_energy  vol_energy  cell_thickness 
     │ String    Number       Number      Number         
─────┼───────────────────────────────────────────────────
   1 │ NMC811     318.0±10.0  963.0±37.0     0.712±0.022
   2 │ LFP         255.6±8.7  637.0±28.0      0.94±0.038


Now we have an NMC811 cell with 318 Wh/kg and 963 Wh/L, and an LFP/Li cell with 255 Wh/kg, and 637 Wh/L. This gives us an LFP/Li cell with very nearly the same energy density, both gravimetric and volumetric, as the NMC811/Gr cell, thanks to removing that very light excess Li metal!

## 6.1 Sulfide electrolyte

We can also consider the sulfide electrolytes, as developed by companies such as Solid Power. Compared to the oxides, the sulfides are less dense, and also mechanically soft enough that they are considered more suitable for all-solid-state batteries with no liquid component. To account for these correctly, we can consider a separator of a perfectly dense sulfide with an assumed density of 2.0 g/cc, and and an electrolyte component (which sits in the cathode pores), with the same density:

In [15]:
sulfide_SE = Separator(name = "Sulfide SE", thickness = 20E-4, porosity = 0, density = 2.0)
sulfide_electrolyte = Electrolyte(salt = "Sulfide electrolyte", solvent = "NA", density = 2.0)

Electrolyte
  salt: String "Sulfide electrolyte"
  solvent: String "NA"
  concentration: Nothing nothing
  density: Float64 2.0
  saltmassfrac: Nothing nothing


Let's assume that the volume fraction of the 'catholyte' component is the same as in the previous LLZO/catholyte case:

In [16]:
result = DataFrame(material = String[], grav_energy = Number[], vol_energy = Number[], 
    cell_thickness = Number[])

for i in 1:nrow(positives)
    
    cell = automotive_pouch(
        positive = Electrode(
            ElectrodeComposite(
                active_material = positives[i,1], 
                areal_cap = 3.3 ± 0.1, 
                active_frac = 0.95 ± 0.02, 
                density = positives[i,2]),
            Al14
        ),
        
        negative = Electrode(
            ElectrodeComposite(
                active_material = Li_metal,          
                areal_cap = 3.3 ± 0.1,               
                active_frac = 1,                    
                density = 0.534),                
            Cu8
        ),
        
        separator = sulfide_SE,           # <---
        electrolyte = sulfide_electrolyte,
        ecap_ratio = (1.7 ± 0.1) * 0.33
    )
    
    push!(result, [positives[i,1].name, gravimetric_energy_anodefree(cell), volumetric_energy(cell),
            thickness(cell)])
    
end

println(result)

2×4 DataFrame
 Row │ material  grav_energy  vol_energy  cell_thickness 
     │ String    Number       Number      Number         
─────┼───────────────────────────────────────────────────
   1 │ NMC811     366.0±12.0  963.0±37.0     0.712±0.022
   2 │ LFP        291.0±10.0  637.0±28.0      0.94±0.038


The result is that the volumetric energy densities are the same (all the thicknesses are identical), but the gravimetric energy is increased, up to 366 Wh/kg for NMC811 and 291 Wh/kg for LFP: 13% higher than the NMC811/Gr baseline.

# 7. Summary

We have looked at modelling several different types of cells - LFP, Li metal, solid state, anode-free; and using batch processing to conveniently test several different cases.

In terms of technical results, we've seen that LFP is typically around 20% lower in gravimetric energy compared to NMC811, if the electrodes are prepared to similar specifications, but lower still in volumetric energy due to a lower bulk density.

However, LFP-Li cells, even solid state with relatively heavy oxide separators, could provide effectively the same energy density as NMC811/Gr cells, but would be expected to be much safer, due to the elimination of several thermal runaway triggers (oxidising cathode, large reduction in flammable components namely separator, graphite, and much of the liquid electrolyte), and could be cheaper. With sulfide electrolytes, they could be higher still.

It's perhaps not surprising it's getting attention...